#  Example 4: Parkhurst and Appelo 2013 (PHREEQC-3 Example 11) 1D Cation Exchange

Cation exchange flushing of a sodium-potassium nitrate solution with calcium chloride.

This modelling example is the first among several examples that include and demonstrate ion-exchanging reactions in flow-through systems. The example was originally used as PHREEQM (Nienhuis et al., 1994) test case, and is also included in the PHREEQC-3 documentation (Parkhurst and Appelo, 2013) as Example 11. Further discussion can be found in (Appelo and Postma, 1993), where it forms Example 10.13, and in (Appelo, 1994). The one-dimensional simulation problem describes a hypothetical column experiment where porewater containing sodium (Na<sup>+</sup>), potassium (K<sup>+</sup>) and nitrate (NO<sup>-</sup><sub>3</sub>) in equilibrium with exchangeable cations is flushed by a calcium chloride (CaCl<sub>2</sub>) solution

# Installation and Setup

This notebook is designed to be run from:   
- A cloned or copied version of this `mf6rtm` respository and 
- A custom [conda](https://docs.conda.io/en/latest/) virtual environment as described in our [Install Development Environment](benchmark/readme.md#install-development-environment) instructions, especially including:
  - [Step 3. Create a Conda Environment for this Repository](benchmark/readme.md#3-create-a-conda-environment-for-this-repository), using the [`environment.yml`](benchmark/environment.yml) file included in this folder.
  - [Step 4. Install this Package in Develop Mode](benchmark/readme.md#4-install-this-package-in-develop-mode), by adding this repository to your conda path.

## Python Imports

In [1]:
import os
from pathlib import Path
import re
import difflib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import flopy

In [2]:
# Import this package
from mf6rtm import mf6rtm, mup3d, utils

### If you get `ModuleNotFoundError`

If you get `ModuleNotFoundError`, you need to install `mf6rtm` into your environment.

If using the development environment, the following steps will install in develop mode:
1. Run the [`conda develop`](https://docs.conda.io/projects/conda-build/en/latest/resources/commands/conda-develop.html) command in your terminal with your local absolute path to the `src` directory of this repo.
2. Restart the kernel.
3. Rerun the import statements above.

In [3]:
# Find project directory (i.e. the parent to `/examples` directory for this notebook)
project_path = Path.cwd().parent
# Your source directory should be: 
src_path = project_path / 'src' 
src_path

PosixPath('/Users/aaufdenkampe/Documents/Python/mf6rtm/src')

### To install in developer mode:
- Confirm that the output of `src_path` points to the MF6RTM `src` directory 
- Uncomment line below
- NOTE 1: The Jupyter [`%conda` magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-conda) runs conda terminal commands directly from this notebook.
- NOTE 2: We can inject Python string objects into magic commmands using a syntax similar to Python's F-strings.

In [4]:
# %conda develop {src_path}

If the path was added, Restart the kernel and rerun the cells above.

## Set Paths to Input and Output Files with `pathlib`

Use the [pathlib](https://docs.python.org/3/library/pathlib.html) library (built-in to Python 3) to manage paths indpendentely of OS or environment.

This blog post describes `pathlib`'s benefits relative to using the `os` library or manual approaches.
- https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f

In [5]:
# Find your current working directory, which should be folder for this notebook.
working_dir = Path.cwd()
working_dir

PosixPath('/Users/aaufdenkampe/Documents/Python/mf6rtm/benchmark')

In [6]:
# Project prefix for inputs and outputs
prefix = 'ex4'

In [7]:
# Phreeqc input file folders
dataws = working_dir / 'data'
databasews = working_dir / 'database'

## Functions to compare outputs from pht3d and mf6rtm

In [8]:
def find_closest_match(query, dictionary):
    closest_match = difflib.get_close_matches(query, dictionary.keys(), n=1)
    if closest_match:
        return closest_match[0]
    else:
        return None
    
def calc_rows_from_ncol(variables, ncols=4):
    """
    Calculates number of rows for subplots
    from ncols and len of variables to plot.

    Parameters:
        variables (list or sequence): list of variables to plot
        ncols (int): number of columns to plot
    """
    n_subplots = len(variables)
    # calculate number of rows
    nrows = n_subplots // ncols + (n_subplots % ncols > 0)
    return nrows

# Flow and Transport Setup

## Model params and setup

In [9]:
# General
length_units = "meters"
time_units = "days"

# Model discretization
nlay = 1  # Number of layers
Lx = 0.08 #m
ncol = 40 # Number of columns
nrow = 1  # Number of rows
delr = Lx/ncol #10.0  # Column width ($m$)
delc = 1.0  # Row width ($m$)
top = 1.  # Top of the model ($m$)
# botm = 0.0  # Layer bottom elevations ($m$)
zbotm = 0.
botm = np.linspace(top, zbotm, nlay + 1)[1:]

#tdis
nper = 1  # Number of periods
tstep = 0.002 *0.5 # Time step ($days$)
perlen = 0.24  # Simulation time ($days$)
nstp = perlen/tstep #100.0
dt0 = perlen / nstp
tdis_rc = []
tdis_rc.append((perlen, nstp, 1.0))

#injection
q = 1 #injection rate m3/d
wel_spd = [[(0,0,0), q]] # well stress period data


#hydraulic properties
prsity = 1 # Porosity
k11 = 1.0  # Horizontal hydraulic conductivity ($m/d$)
k33 = k11  # Vertical hydraulic conductivity ($m/d$)
strt = np.ones((nlay, nrow, ncol), dtype=float)*1

#chd: Time-Variant Specified Head
r_hd = 1
strt = np.ones((nlay, nrow, ncol), dtype=float)

chdspd = [[(i, 0, ncol-1), r_hd] for i in range(nlay)] # Constant head boundary $m$

#transport
dispersivity = 0.002 # Longitudinal dispersivity ($m$)
disp_tr_vert = dispersivity*0.1 # Transverse vertical dispersivity ($m$)

icelltype = 1  # Cell conversion type

# Set solver parameter values (and related)
nouter, ninner = 300, 600
hclose, rclose, relax = 1e-6, 1e-6, 1.0

## Initialize Chemistry in Domain

These functions:
- read PHREEQC inputs by PHREEQC "keyword data blocks"
- convert to a dictionary
- instantiate `mup3d.{Block}` classes that contain the block's geochemical components
- set the grid size/shape for the components

### SOLUTION Block

In [10]:
# Get list of phreeqc input files for this example
files = [f for f in os.listdir(dataws) if f.startswith(prefix)]
files

['ex4_solutions.csv', 'ex4_exchange.csv', 'ex4_postfix.phqr']

In [11]:
# Read solutions file, containing initial conditions and boundary conditions
solutions_filepath = dataws /f"{prefix}_solutions.csv"

solutionsdf = pd.read_csv(
    solutions_filepath, 
    comment = '#',  
    index_col = 0,
)
solutionsdf

,sol_bck,sol_inflow
comp,,
pH,7.0000,7.0000
pe,12.5000,12.5000
Ca,0.0000,0.0006
Cl,0.0000,0.0012
K,0.0002,0.0000
Na,0.0010,0.0000
N(+5),0.0012,0.0000


In [12]:
#add data to the mup3d class
solutions = utils.solution_df_to_dict(solutionsdf)
solution = mup3d.Solutions(solutions)
solution.data

{'pH': [7.0, 7.0],
 'pe': [12.5, 12.5],
 'Ca': [0.0, 0.0006],
 'Cl': [0.0, 0.0012],
 'K': [0.0002, 0.0],
 'Na': [0.001, 0.0],
 'N(+5)': [0.0012, 0.0]}

In [13]:
solution.names

['pH', 'pe', 'Ca', 'Cl', 'K', 'Na', 'N(+5)']

In [14]:
#preallocate initial conditions sollutions grid
sol_ic = np.ones((nlay, nrow, ncol), dtype=float)
solution.set_ic(sol_ic)
solution.ic

array([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.]]])

In [15]:
solution.ic.shape

(1, 1, 40)

### EXCHANGE Block

In [16]:
exchange_filepath = dataws /f"{prefix}_exchange.csv"
exchange_df = pd.read_csv(exchange_filepath, comment = '#',  index_col = 0)
exchange_dict = utils.solution_df_to_dict(exchange_df)

exchanger = mup3d.ExchangePhases(exchange_dict)
exchanger.set_ic(np.ones((nlay, nrow, ncol), dtype=float))

In [17]:
exchanger.data

{'CaX2': [0.0], 'NaX': [0.0005493], 'KX': [0.0005507]}

In [18]:
exchanger.ic

array([[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.]]])

## Create `Mup3d` model class

In [19]:
#create model class
model = mup3d.Mup3d(prefix,solution, nlay, nrow, ncol)

#set model workspace
model.set_wd(working_dir / f'{prefix}' / 'mf6rtm')

#set Phreeqc database
database_filepath = working_dir / databasews / 'pht3d_datab.dat'
model.set_database(database_filepath)

model.set_exchange_phases(exchanger)

#get Phreeqc postfix file
postfix = dataws /  f'{prefix}_postfix.phqr'
model.set_postfix(postfix)

In [20]:
model.name

'ex4'

### Initialize `Mup3d` model class
This creates a PhreeqcRM instance based on components in Chemistry Blocks and initial conditions then calculates inital speciation.

In [21]:
# Intializing the mup3d class calculates the equilibrated 
# initial concentration array
model.initialize()

Using temperatue of 25.0 for all cells
Phreeqc initialized


In [22]:
model.components

['H', 'O', 'Charge', 'Ca', 'Cl', 'K', 'N', 'Na']

In [23]:
model.init_conc_array_phreeqc.shape

(320,)

In [24]:
# 1D Array of concentrations (mol/L) structured for PhreeqcRM, 
# with each component conc for each grid cell
# ordered by `model.components`
model.init_conc_array_phreeqc

array([ 1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        1.10684171e+02,  1.10684171e+02,  1.10684171e+02,  1.10684171e+02,
        5.53456749e+01,  5.53456749e+01,  5.53456749e+01,  5.53456749e+01,
        5.53456749e+01,  5.53456749e+01,  5.53456749e+01,  5.53456749e+01,
        5.53456749e+01,  5.53456749e+01,  5.53456749e+01,  5.53456749e+01,
        5.53456749e+01,  

In [25]:
# Dictionary of concentrations in units of moles per m^3 
# and structured to match the shape of Modflow's grid
model.sconc

{'H': array([[[110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486,
          110684.1711486, 110684.1711486, 110684.1711486, 110684.1711486]]]),
 'O': array([[[55345.67494365, 55345.67494365, 55345.67494365, 55345.67494365,
          55345.67494365, 55345.67494365, 55345.67494365, 55345.67494365,
          55345.67494365, 55345.67494365, 55345.67494365, 55345.67494365,
          55345.67494365

In [26]:
model.sconc['H'].shape

(1, 1, 40)

In [27]:
from importlib import reload
reload(mup3d)

<module 'mf6rtm.mup3d' from '/Users/aaufdenkampe/Documents/Python/mf6rtm/src/mf6rtm/mup3d.py'>

## Initialize Inflow Chemistry

In [28]:
wellchem = mup3d.ChemStress('wel')

# column number(s) for stress period data in `solution.data` 
sol_spd = [2] 
wellchem.set_spd(sol_spd)
wellchem.sol_spd

[2]

In [29]:
# Values for different wells could be retrieved similar to this example
for data_column_number in wellchem.sol_spd:
     solutions_list_index = data_column_number - 1
     for key, value in solution.data.items():
        print(key, value[solutions_list_index])

pH 7.0
pe 12.5
Ca 0.0006
Cl 0.0012
K 0.0
Na 0.0
N(+5) 0.0


In [30]:
# Set and initialize stress period chemical concentrations for each well
model.set_chem_stress(wellchem)

Initializing ChemStress
[-1, -1, -1, -1, -1, -1, -1]
0 2
[2, -1, -1, -1, -1, -1, -1]
ChemStress wel initialized


In [38]:
# Equilbrated concentrations for Solution 1
model.wel.data

{0: [np.float64(110684.17114503868),
  np.float64(55342.085589753035),
  np.float64(-1.3109877640744503e-06),
  np.float64(0.5982258070710701),
  np.float64(1.1964516141421362),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0)]}

In [31]:
# Component names
model.wel.auxiliary

['H', 'O', 'Charge', 'Ca', 'Cl', 'K', 'N', 'Na']

In [32]:
# Well Stress Period Data was defined in the 'Flow and Transport Setup' Section,
# with the well location (xyz), and flow rate (q)
wel_spd

[[(0, 0, 0), 1]]

In [33]:
# Append Conc data to Well Stress Period Data list, 
# NOTE: only run this once
for i in range(len(wel_spd)):
    wel_spd[i].extend(model.wel.data[i])
wel_spd

[[(0, 0, 0),
  1,
  np.float64(110684.17114503868),
  np.float64(55342.085589753035),
  np.float64(-1.3109877640744503e-06),
  np.float64(0.5982258070710701),
  np.float64(1.1964516141421362),
  np.float64(0.0),
  np.float64(0.0),
  np.float64(0.0)]]

# Build MF6 Model

Using [flopy](https://flopy.readthedocs.io), with attributes from our Mup3d model instance, along with other attributes defined above

In [ ]:
def build_model(mup3d):

    #####################        GWF model           #####################
    gwfname = 'gwf'
    sim_ws = mup3d.wd
    sim = flopy.mf6.MFSimulation(sim_name=mup3d.name, sim_ws=sim_ws, exe_name='mf6')

    # Instantiating MODFLOW 6 time discretization
    flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units)

    # Instantiating MODFLOW 6 groundwater flow model
    gwf = flopy.mf6.ModflowGwf(
        sim,
        modelname=gwfname,
        save_flows=True,
        model_nam_file=f"{gwfname}.nam",
    )

    # Instantiating MODFLOW 6 solver for flow model
    imsgwf = flopy.mf6.ModflowIms(
        sim,
        complexity="complex",
        print_option="SUMMARY",
        outer_dvclose=hclose,
        outer_maximum=nouter,
        under_relaxation="NONE",
        inner_maximum=ninner,
        inner_dvclose=hclose,
        rcloserecord=rclose,
        linear_acceleration="CG",
        scaling_method="NONE",
        reordering_method="NONE",
        relaxation_factor=relax,
        filename=f"{gwfname}.ims",
    )
    sim.register_ims_package(imsgwf, [gwf.name])

    # Instantiating MODFLOW 6 discretization package
    dis = flopy.mf6.ModflowGwfdis(
        gwf,
        length_units=length_units,
        nlay=nlay,
        nrow=nrow,
        ncol=ncol,
        delr=delr,
        delc=delc,
        top=top,
        botm=botm,
        idomain=np.ones((nlay, nrow, ncol), dtype=int),
        filename=f"{gwfname}.dis",
    )
    dis.set_all_data_external()

    # Instantiating MODFLOW 6 node-property flow package
    npf = flopy.mf6.ModflowGwfnpf(
        gwf,
        save_flows=True,
        save_saturation = True,
        icelltype=icelltype,
        k=k11,
        k33=k33,
        save_specific_discharge=True,
        filename=f"{gwfname}.npf",
    )
    npf.set_all_data_external()
    # sto = flopy.mf6.ModflowGwfsto(gwf, ss=1e-6, sy=0.25)

    # Instantiating MODFLOW 6 initial conditions package for flow model
    flopy.mf6.ModflowGwfic(gwf, strt=strt, filename=f"{gwfname}.ic")
    
    wel = flopy.mf6.ModflowGwfwel(
            gwf,
            stress_period_data=wel_spd,
            save_flows = True,
            auxiliary = model.components,
            pname = 'wel',
            filename=f"{gwfname}.wel"
        )
    wel.set_all_data_external()

    # Instantiating MODFLOW 6 constant head package
    chd = flopy.mf6.ModflowGwfchd(
        gwf,
        maxbound=len(chdspd),
        stress_period_data=chdspd,
        # auxiliary=mup3d.components,
        save_flows=False,
        pname="CHD",
        filename=f"{gwfname}.chd",
    )
    chd.set_all_data_external()

    # Instantiating MODFLOW 6 output control package for flow model
    oc_gwf = flopy.mf6.ModflowGwfoc(
        gwf,
        head_filerecord=f"{gwfname}.hds",
        budget_filerecord=f"{gwfname}.cbb",
        headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
        saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
        printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
    )
    
    #####################           GWT model          #####################
    for c in mup3d.components:
        print(f'Setting model for component: {c}')
        gwtname = c
        
        # Instantiating MODFLOW 6 groundwater transport package
        gwt = flopy.mf6.MFModel(
            sim,
            model_type="gwt6",
            modelname=gwtname,
            model_nam_file=f"{gwtname}.nam"
        )

        # create iterative model solution and register the gwt model with it
        print('--- Building IMS package ---')
        imsgwt = flopy.mf6.ModflowIms(
            sim,
            print_option="SUMMARY",
            outer_dvclose=hclose,
            outer_maximum=nouter,
            under_relaxation="NONE",
            inner_maximum=ninner,
            inner_dvclose=hclose,
            rcloserecord=rclose,
            linear_acceleration="BICGSTAB",
            scaling_method="NONE",
            reordering_method="NONE",
            relaxation_factor=relax,
            filename=f"{gwtname}.ims",
        )
        sim.register_ims_package(imsgwt, [gwt.name])

        print('--- Building DIS package ---')
        dis = gwf.dis

        # create grid object
        dis = flopy.mf6.ModflowGwtdis(
            gwt,
            length_units=length_units,
            nlay=nlay,
            nrow=nrow,
            ncol=ncol,
            delr=delr,
            delc=delc,
            top=top,
            botm=botm,
            idomain=np.ones((nlay, nrow, ncol), dtype=int),
            filename=f"{gwtname}.dis",
        )
        dis.set_all_data_external()

         
        ic = flopy.mf6.ModflowGwtic(gwt, strt=mup3d.sconc[c], filename=f"{gwtname}.ic")
        ic.set_all_data_external()
        
        # Instantiating MODFLOW 6 transport source-sink mixing package
        sourcerecarray = ['wel', 'aux', f'{c}']
        # sourcerecarray = [()]
        ssm = flopy.mf6.ModflowGwtssm(
            gwt, 
            sources=sourcerecarray, 
            save_flows=True,
            print_flows=True,

            filename=f"{gwtname}.ssm"
        )
        ssm.set_all_data_external()
        # Instantiating MODFLOW 6 transport adv package
        print('--- Building ADV package ---')
        adv = flopy.mf6.ModflowGwtadv(
            gwt,
            scheme="tvd",
        )

        # Instantiating MODFLOW 6 transport dispersion package
        alpha_l = np.ones(shape=(nlay, nrow, ncol))*dispersivity  # Longitudinal dispersivity ($m$)
        ath1 = np.ones(shape=(nlay, nrow, ncol))*dispersivity*0.1 # Transverse horizontal dispersivity ($m$)
        atv = np.ones(shape=(nlay, nrow, ncol))*dispersivity*0.1   # Transverse vertical dispersivity ($m$)

        print('--- Building DSP package ---')
        dsp = flopy.mf6.ModflowGwtdsp(
            gwt,
            xt3d_off=True,
            alh=alpha_l,
            ath1=ath1,
            atv = atv,
            # diffc = diffc,
            filename=f"{gwtname}.dsp",
        )
        dsp.set_all_data_external()

        # Instantiating MODFLOW 6 transport mass storage package (formerly "reaction" package in MT3DMS)
        print('--- Building MST package ---')

        first_order_decay = None

        mst = flopy.mf6.ModflowGwtmst(
            gwt,
            porosity=prsity,
            first_order_decay=first_order_decay,
            filename=f"{gwtname}.mst",
        )
        mst.set_all_data_external()

        print('--- Building OC package ---')

        # Instantiating MODFLOW 6 transport output control package
        oc_gwt = flopy.mf6.ModflowGwtoc(
            gwt,
            budget_filerecord=f"{gwtname}.cbb",
            concentration_filerecord=f"{gwtname}.ucn",
            concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 10, "GENERAL")
                                        ],
            saverecord=[("CONCENTRATION", "ALL"), 
                        ("BUDGET", "ALL")
                        ],
            printrecord=[("CONCENTRATION", "ALL"), 
                            ("BUDGET", "ALL")
                            ],
        )

        # Instantiating MODFLOW 6 flow-transport exchange mechanism
        flopy.mf6.ModflowGwfgwt(
            sim,
            exgtype="GWF6-GWT6",
            exgmnamea=gwfname,
            exgmnameb=gwtname,
            filename=f"{gwtname}.gwfgwt",
        )

    sim.write_simulation()
    utils.prep_bins(sim_ws, src_path=os.path.join('..','bin'))
    
    return sim

In [ ]:
sim = build_model(model)

In [ ]:
# Run the model using this wrapper function for `mf6rtm.solve(model.wd)`
model.run()

# Figures

Plot MF6RTM results versus results from PHT3D and PHREEQC.

## Read Results from 3 models

In [ ]:
## Read pht3d results

wd = os.path.join(f'{prefix}', f'pht3d')
# dx = 0.01
simdf = pd.read_csv(os.path.join(wd, 'out.dat'), sep = '\t', skipinitialspace=True, index_col=[0])
simdf.drop(simdf.columns[len(simdf.columns)-1], axis=1, inplace=True)

simdf.loc[:, 'x'] = simdf['cell'] * delr

simapi = pd.read_csv(os.path.join(model.wd,'sout.csv'), sep = ',', skipinitialspace=True, index_col=[0])
simapi.loc[:, 'x'] = (simapi['cell'] + 1)*delr

#offset index of simapi by tstep
simapi.index = simapi.index + tstep

simapi

# get all ucn files in wd
ucn_files = [f for f in os.listdir(wd) if f.lower().endswith('.ucn')]
ucn_files

# get file that ends in py
pht3dpy = [f for f in os.listdir(wd) if f.endswith('py')]

#read pht3dpy file
ucndic = {}
pht3dpy = os.path.join(wd, pht3dpy[0])
with open(pht3dpy, 'r') as f:
    # print(f.read())
    for l in f:
        n =re.findall(r'\d+', l.split()[-1])[-1]
        ucndic[l.split()[0].replace('_', "")] = f"PHT3D{n}.UCN"
        
for k,v in ucndic.items():
    ucn = flopy.utils.HeadFile(os.path.join(wd, v),text=f"concentration")
    results = ucn.get_alldata()
    ucndic[k] = results
timespht3d = ucn.get_times()
    
for var in simapi.columns:
    closest_match = find_closest_match(var, ucndic)
    if closest_match:
        ucndic[var] = ucndic.pop(closest_match)


In [ ]:
## Read mf6rtm results

wd = os.path.join(f'{prefix}',f'mf6rtm')
# get all ucn files in wd
ucn_files = [f for f in os.listdir(wd) if f.lower().endswith('.ucn')]
ucn_files

ucndic_mf6 = {}
for k in ucn_files:
    ucn = flopy.utils.HeadFile(os.path.join(wd, k),text=f"concentration")
    results = ucn.get_alldata()
    ucndic_mf6[f'{k.split(".")[0]}'] = results
times = ucn.get_times()

In [ ]:
## Read phreeqc results

wd = os.path.join(f'{prefix}',f'phreeqc')
# dx = 0.01
phreeqcout = pd.read_csv(os.path.join(wd, 'ex11trn.csv'), sep = '\t', skipinitialspace=True, index_col=[0])
phreeqcout.drop(phreeqcout.columns[len(phreeqcout.columns)-1], axis=1, inplace=True)

phreeqcout.loc[:, 'x'] = phreeqcout['cell'] * delr

## Create plots

In [ ]:
### Plot Effluent Concentrations, similar to Phreeqc3 Manual's Example 11, Figure 12B.

pncol=2
variables = list(simapi.iloc[:,1:-1 ].columns) #dissolved only

pnrow = calc_rows_from_ncol(variables, pncol)

mf6df = simapi[simapi['cell'] == simapi['cell'].max()].copy()
pht3df = simdf[simdf['cell'] == simdf['cell'].max()].copy()
phreeqdf = phreeqcout[phreeqcout['cell'] == phreeqcout['cell'].max()].copy()

colors = ['purple', 'r', 'g', 'b']

fig, axs = plt.subplots(1,1, figsize = (6.3, 6.3))
ax = axs
[ax.plot(mf6df.index, mf6df.loc[:, var]*1000, label = f"{var} mf6rtm", c = colors[variables.index(var)]) for var in variables];
[ax.scatter(phreeqdf.time_d[::4], phreeqdf.loc[:, var][::4]*1000, label = f"{var} phreeqc",  marker = 's',facecolor = 'None', edgecolors = colors[variables.index(var)]) for var in variables];
[ax.scatter(pht3df.index[::4], pht3df.loc[:, var][::4]*1000, label = f"{var} pht3d",  facecolor = 'None', edgecolors = colors[variables.index(var)]) for var in variables];

ax.set_ylabel('C (mmol L$^{-3}$)')
ax.set_xlabel('Time (d)')
ax.legend()
fig.tight_layout()
fig.savefig(os.path.join(f'{prefix}.png'), dpi = 300)

In [ ]:
### Detailed Comparison Plots

#get common keys from ucndic and ucndic_mf6
common_keys = set(ucndic.keys()).intersection(ucndic_mf6.keys())
common_keys

pncol=2
variables = common_keys

pnrow = calc_rows_from_ncol(variables, pncol)



fig, axs = plt.subplots(pnrow,pncol, figsize = (6.3, 6.3))
for var, ax in zip(common_keys, axs.flatten()):
    ax.plot([x for x in timespht3d], ucndic[var][:,0,0,-1], label = f'PHT3D')
    ax.plot([x for x in times[:]], ucndic_mf6[var][:,0,0,-1]/1000, label = f'mf6rtm')

    #get min and max of y axis
    xmin, xmax = ax.get_ylim()
    # ax.set_ylim(xmin*.8, xmax*1.2)

    ax.set_xlabel('time (days)')
    if var not in ['pH', 'pe']:
        ax.set_ylabel('C (mol L3$^{-3}$)')
    ax.set_title(f'{var}')
    ax.ticklabel_format(useOffset=False)
    ax.legend()
fig.tight_layout()